<a href="https://colab.research.google.com/github/Max-Konovalov/mtuci_lab1/blob/main/mtuci_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ссылка на гитхаб https://github.com/Max-Konovalov/mtuci_lab1

import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib
#импорт необходимых библиотек

In [ ]:
#Подключение CSV файла и вывод информации первых 10 строк
drive.mount('/content/drive/')
file_path = '/content/Laptop_price.csv'
df = pd.read_csv(file_path)
print(df.head(10))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
    Brand  Processor_Speed  RAM_Size  Storage_Capacity  Screen_Size    Weight  \
0    Asus         3.830296        16               512    11.185147  2.641094   
1    Acer         2.912833         4              1000    11.311372  3.260012   
2  Lenovo         3.241627         4               256    11.853023  2.029061   
3    Acer         3.806248        16               512    12.280360  4.573865   
4    Acer         3.268097        32              1000    14.990877  4.193472   
5      HP         1.881348        16               256    11.943958  4.840268   
6  Lenovo         2.940721        32               256    11.713246  4.344846   
7  Lenovo         3.016788         4               256    15.588315  2.941120   
8  Lenovo         2.560327         4               512    15.026332  2.743620   
9    Acer         3.341111        32               512    1

In [ ]:
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print(X)

      Brand  Processor_Speed  RAM_Size  Storage_Capacity  Screen_Size  \
0      Asus         3.830296        16               512    11.185147   
1      Acer         2.912833         4              1000    11.311372   
2    Lenovo         3.241627         4               256    11.853023   
3      Acer         3.806248        16               512    12.280360   
4      Acer         3.268097        32              1000    14.990877   
..      ...              ...       ...               ...          ...   
995      HP         3.343584         4              1000    12.587095   
996    Dell         2.780555         8               256    12.679356   
997    Dell         3.200569         4               512    12.666315   
998    Asus         1.604182         8               256    11.215581   
999  Lenovo         1.711980         4               256    16.561498   

       Weight  
0    2.641094  
1    3.260012  
2    2.029061  
3    4.573865  
4    4.193472  
..        ...  
995  3.1623

In [ ]:
print(y)

0      17395.093065
1      31607.605919
2       9291.023542
3      17436.728334
4      32917.990718
           ...     
995    31593.668017
996     9149.521832
997    16552.404779
998     9407.473459
999     8807.696702
Name: Price, Length: 1000, dtype: float64


In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor),
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, '/content/laptop_price_model.pkl')



['/content/laptop_price_model.pkl']

In [ ]:
!git init
#Прописал в .gitignore файле /drive (на диске лежат некоторые рабочие файлы, так что не могу работать через него)

Reinitialized existing Git repository in /content/.git/


In [ ]:
!git add .

In [ ]:
!git commit -m "Some message"

On branch main
nothing to commit, working tree clean


In [ ]:
# Add your email to the global config-file
!git config --global user.email MaxKOYT@yandex,ru

# Add your email to the global config-file
!git config --global user.name Max-Konovalov

In [ ]:
!git remote add origin https://github.com/Max-Konovalov/mtuci_lab1.git

In [ ]:
!git push -u origin master

error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/Max-Konovalov/mtuci_lab1.git'


In [ ]:
!git remote rm origin

In [ ]:
%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

# Загрузка обученной модели
model_path = "/content/drive/MyDrive/laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}


Writing app.py


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok config add-authtoken "2uiMzixdQZZMvYdQZL3aar3JL1B_3f3WiMsGNneJ3JvSpAEZN"

#Далее в фоновом режиме запускаем uvicorn:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &


#После чего уже запускаем FastAPI

from pyngrok import ngrok
# Подключаем публичный URL
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
API доступно по адресу: NgrokTunnel: "https://aa01-35-196-18-66.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

# Загрузка данных
file_path = '/content/Laptop_price.csv'
try:
    df = pd.read_csv(file_path)
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")

# 1. Проверка CSV на уязвимости (CSV Injection)
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].astype(str).apply(lambda x: x.lstrip().startswith(dangerous_chars) or any(x.startswith(f' {c}') for c in dangerous_chars)).any():
            print(f"Обнаружены потенциальные CSV-инъекции в столбце {col}!")
        else:
            print(f"Столбец {col} безопасен.")

check_csv_injection(df)

# 2. Фильтрация данных от SQL-инъекций и XSS-атак
def clean_input(value):
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION", "--"]
    xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror=.*']

    for keyword in sql_keywords:
        if keyword.lower() in value.lower():
            return "[BLOCKED]"

    for pattern in xss_patterns:
        if re.search(pattern, value, re.IGNORECASE):
            return "[BLOCKED]"

    return value

df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")

# 3. Хеширование столбца с ценами (SHA-256)
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

df['Price_Hashed'] = df['Price'].apply(hash_price)
print("Столбец с хешированными ценами добавлен.")

# 4. Шифрование данных (например, цены ноутбуков)
key_price = Fernet.generate_key()
cipher_price = Fernet(key_price)

def encrypt_price(price):
    return cipher_price.encrypt(str(price).encode()).decode()

def decrypt_price(encrypted_price):
    return cipher_price.decrypt(encrypted_price.encode()).decode()

df['Price_Encrypted'] = df['Price'].apply(encrypt_price)
print("Столбец с зашифрованными ценами добавлен.")

# 5. Шифрование столбца RAM
key_ram = Fernet.generate_key()
cipher_ram = Fernet(key_ram)

def encrypt_ram(ram):
    return cipher_ram.encrypt(str(ram).encode()).decode()

def decrypt_ram(encrypted_ram):
    return cipher_ram.decrypt(encrypted_ram.encode()).decode()

df['RAM_Encrypted'] = df['RAM_Size'].apply(encrypt_ram)
print("Столбец с зашифрованной RAM добавлен.")

# Вывод 5 расшифрованных значений RAM
print("Примеры расшифрованных значений RAM:")
for encrypted_ram in df['RAM_Encrypted'].head(5):
    print(decrypt_ram(encrypted_ram))

# Сохранение обработанных данных
output_path = "Laptop_price_secured.csv"
df.to_csv(output_path, index=False)
print(f"Обработанный файл сохранен: {output_path}")


Столбец Brand безопасен.
Фильтрация данных завершена.
Столбец с хешированными ценами добавлен.
Столбец с зашифрованными ценами добавлен.
Столбец с зашифрованной RAM добавлен.
Примеры расшифрованных значений RAM:
16
4
4
16
32
Обработанный файл сохранен: Laptop_price_secured.csv


<ipython-input-26-650552b0f983>:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
